#### LeetCode - SQL - #1211

Refer [1211. Queries Quality and Percentage](https://leetcode.com/problems/queries-quality-and-percentage/description/)

We define query  `quality`  as:

> The average of the ratio between query rating and its position.

We also define  `poor query percentage`  as:

> The percentage of all queries with rating less than 3.

Write a solution to find each  `query_name`, the  `quality`  and  `poor_query_percentage`.

Both  `quality`  and  `poor_query_percentage`  should be  **rounded to 2 decimal places**.

Return the result table in  **any order**.

In [0]:
data = [['Dog', 'Golden Retriever', 1, 5], ['Dog', 'German Shepherd', 2, 5], ['Dog', 'Mule', 200, 1], ['Cat', 'Shirazi', 5, 2], ['Cat', 'Siamese', 3, 3], ['Cat', 'Sphynx', 7, 4]]

columns = ['query_name', 'result', 'position', 'rating']
schema = {'query_name':'object', 'result':'object', 'position':'Int64', 'rating':'Int64'}

In [0]:
import pandas

pandas_dataframe = pandas.DataFrame(data=data, columns=columns).astype(schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
queries_dataframe = spark_context.createDataFrame(pandas_dataframe)
queries_dataframe.printSchema()
queries_dataframe.show()

root
 |-- query_name: string (nullable = true)
 |-- result: string (nullable = true)
 |-- position: long (nullable = true)
 |-- rating: long (nullable = true)

+----------+----------------+--------+------+
|query_name|          result|position|rating|
+----------+----------------+--------+------+
|       Dog|Golden Retriever|       1|     5|
|       Dog| German Shepherd|       2|     5|
|       Dog|            Mule|     200|     1|
|       Cat|         Shirazi|       5|     2|
|       Cat|         Siamese|       3|     3|
|       Cat|          Sphynx|       7|     4|
+----------+----------------+--------+------+



#### Miscellanous DataFrame Functions

In [0]:
queries_dataframe.columns #Prints a list of column names
queries_dataframe.dtypes
queries_dataframe.schema

queries_dataframe.head()
queries_dataframe.head(5)
queries_dataframe.first()
queries_dataframe.take(2)

queries_dataframe.count()
queries_dataframe.distinct().count()

queries_dataframe.describe()
queries_dataframe.explain() #Prints logical and physical plans

StructType([StructField('query_name', StringType(), True), StructField('result', StringType(), True), StructField('position', LongType(), True), StructField('rating', LongType(), True)])

In [0]:
from pyspark.sql.functions import avg, round, when

queries_summary_dataframe = queries_dataframe \
        .filter(queries_dataframe.query_name.isNotNull()) \
        .groupBy(queries_dataframe.query_name) \
        .agg(
            round(avg(queries_dataframe.rating/ queries_dataframe.position), 2).alias("quality"),
            round(avg(when(queries_dataframe.rating < 3, queries_dataframe.rating).otherwise(None)) * 100, 2).alias("poor_query_percentage")
        )
queries_summary_dataframe.show()

+----------+-------+---------------------+
|query_name|quality|poor_query_percentage|
+----------+-------+---------------------+
|       Dog|    2.5|                100.0|
|       Cat|   0.66|                200.0|
+----------+-------+---------------------+

